## 6. create sentiment change csv file
To test if the sentiment score **changes** for negative tweets after American Airline replys to the conversation that contains **cancel**, **delay** or **connenction**.  

**input**  : database core.db  
**output** : csv file 'obj/word_sentiment_change.csv' -> contains conversation id, user first sentiment score, user last sentiment score

In [ ]:
import sqlite3
import time
import pandas as pd

conn = sqlite3.connect('core.db')

In [ ]:
query = '''select conversation_id
from twitter,conversation
where twitter.id = conversation.tweet_id and twitter.user_id like '22536055' and (twitter.text like '%delay%' or twitter.text like '%cancel%' or twitter.text like '%connection%') '''
query = pd.read_sql_query(query,conn)

In [ ]:
print('unique conversations :',len(query.conversation_id.unique()))

In [ ]:
conversation_id_all = list(query.conversation_id.unique())

In [ ]:
query = '''select created_at,sentiment_score,conversation_id,user_id
from twitter,conversation
where twitter.id = conversation.tweet_id
'''
all_cov = pd.read_sql_query(query,conn)

In [ ]:
st = time.time()
all_cov['created_at'] = all_cov['created_at'].apply(lambda x : pd.to_datetime(x))

print(round(time.time()-st,2),'sec')

In [ ]:
result_df = pd.DataFrame(columns = ['conversation','first','last'])

unique_cov = all_cov.conversation_id.unique()
result_lst = []
# gp = all.groupby('conversation_id')

st = time.time()
for index in conversation_id_all:
    gp_sort = all_cov.groupby('conversation_id').get_group(index).sort_values('created_at')
    user_first_id = gp_sort.iloc[0].user_id
    user_first_senti =  gp_sort.iloc[0].sentiment_score
    if user_first_senti < 0:
        user_last_senti = gp_sort[gp_sort['user_id'] == user_first_id].iloc[-1].sentiment_score
        result_lst = result_lst + [[index,user_first_senti,user_last_senti]]
    
#     display(all_cov.groupby('conversation_id').get_group(index))
#     display(gp_sort)
#     display(gp_sort[gp_sort['user_id'] == user_first_id])
#     print(result_lst)
#     break
print(round(time.time()-st,2),'sec')

In [ ]:
for i in range(len(result_lst)-1):
    result_df.loc[i] = result_lst[i]
result_df.to_csv('obj/word_sentiment_change.csv')

In [ ]:
print('Done')